In [1]:
import dgl
import torch

graph_list, _ = dgl.load_graphs('../save3/graph_vfc.graph')
graph = graph_list[0]
print(graph)

Using backend: pytorch


Graph(num_nodes={'author': 3306925, 'country': 54, 'field': 142726, 'org': 379, 'paper': 5319925, 'topic': 16081, 'venue': 46347},
      num_edges={('country', 'has', 'org'): 432, ('field', 'has', 'venue'): 29480166, ('org', 'has', 'author'): 7209864, ('paper', 'cites', 'paper'): 25626821, ('paper', 'written', 'author'): 13024047, ('topic', 'contains', 'paper'): 13160952, ('venue', 'publishes', 'paper'): 3278873},
      metagraph=[('country', 'org', 'has'), ('org', 'author', 'has'), ('field', 'venue', 'has'), ('venue', 'paper', 'publishes'), ('paper', 'paper', 'cites'), ('paper', 'author', 'written'), ('topic', 'paper', 'contains')])


In [2]:
in_degree = graph.in_degrees(etype='cites')
print(f'in_degree: {in_degree.shape}, {torch.sum(in_degree)}, {in_degree}')
idx = torch.where(in_degree==0)[0]
len(idx)

in_degree: torch.Size([5319925]), 25626821, tensor([10,  0,  0,  ...,  0,  0,  0])


3356390

In [3]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'in_degree':np.array(in_degree)})
df.describe()

,in_degree
count,5.319925e+06
mean,4.817140e+00
std,3.525899e+01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,2.000000e+00
max,1.650200e+04


In [4]:
graph.nodes['paper'].data['citation'] = in_degree

In [17]:
top_nodes = graph.nodes('topic')
pap_nodes = graph.nodes('paper')
print(f'top_nodes:{len(top_nodes)}, {top_nodes}')
print(f'pap_nodes:{len(pap_nodes)}, {pap_nodes}')

top_nodes:16081, tensor([    0,     1,     2,  ..., 16078, 16079, 16080])
pap_nodes:5319925, tensor([      0,       1,       2,  ..., 5319922, 5319923, 5319924])


In [12]:
top_src, pap_dst = graph.edges(etype = 'contains', order='srcdst')
print(f'top_src:{len(top_src)}, {top_src}')
print(f'pap_dst:{len(pap_dst)}, {pap_dst}')

top_src:13160952, tensor([    0,     0,     0,  ..., 16079, 16079, 16080])
pap_dst:13160952, tensor([  82785,  143234,  144174,  ..., 5313841, 5319697,    7644])


In [7]:
citation = graph.nodes['paper'].data['citation']
print(f'citation:{len(citation)}, {citation}')

citation:5319925, tensor([10,  0,  0,  ...,  0,  0,  0])


In [16]:
from tqdm import trange, tqdm

def calc_h_index(cites):
    c = torch.sort(cites, descending=True)[0]
    indx = torch.arange(len(c))
    delta = c-indx
    d = torch.where(delta>0)[0]
    if len(d)>0:
        h_index = d[-1]+1
    else:
        h_index = 0
    return h_index

print(f'top_nodes:{len(top_nodes)}, {top_nodes}')
h_index = torch.zeros(len(top_nodes))
outer = range(len(top_nodes))
for i in outer:
    top = top_nodes[i]
    idx = torch.where(top_src==top)[0]
    src, dst = top_src[idx], pap_dst[idx]

    # for j in dst:
    #     nids = torch.where(top_nodes==dst)

    cites = citation[dst]
    print(f'src:{len(src)}, {src}')
    print(f'dst:{len(dst)}, {dst}')
    print(f'cites:{len(cites)}, {cites}')
    h_index[i] = calc_h_index(cites)
    # print(h)
    break


top_nodes:16081, tensor([    0,     1,     2,  ..., 16078, 16079, 16080])
src:1384, tensor([0, 0, 0,  ..., 0, 0, 0])
dst:1384, tensor([  82785,  143234,  144174,  ..., 5313358, 5313372, 5313374])
cites:1384, tensor([14,  1,  0,  ...,  0,  0,  0])


In [ ]:
graph.nodes['topic'].data['h_index'] = h_index
print(graph.nodes['topic'])

NodeSpace(data={'h_index': tensor([11.,  1.,  2.,  ...,  0., 14.,  1.])})


In [ ]:
au_in = graph.in_degrees(etype='written')
print(f'au_in:{len(au_in)}, {au_in}')

au_in:3306925, tensor([ 8,  7,  1,  ..., 42,  5,  4])


In [ ]:
author_ids = torch.where(au_in >= 50)[0]
print(f'author_ids:{len(author_ids)}, {author_ids}')

author_ids:36288, tensor([    254,     255,     874,  ..., 3302961, 3304332, 3306903])
